<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Agent with Function Tools

In this section, we show how to build an agent with access to arbitrary functions as tools. These functions can be built to perform an arbitrary task.


Lets first install the necessary Python packages for our "Introduction to AI Agents" tutorial. The %pip install command is used to install the following packages:

- **llama-index:** This package provides tools and utilities for building and managing indexes, which are essential for efficient data retrieval and query processing in AI applications.

- **llama-index-llms-openai**: This package integrates the Llama Index with OpenAI's large language models (LLMs). It allows you to leverage the capabilities of OpenAI's advanced language models for various NLP tasks such as text generation, summarization, and more.

- **llama-index-readers-file:** This package includes readers for handling file-based data sources. It enables the indexing and retrieval of data from various file formats, making it easier to manage and process large datasets stored in files.

- **llama-index-embeddings-openai:** This package is used to integrate OpenAI's embedding models with the Llama Index. Embeddings are numerical representations of text that capture semantic meaning, which are crucial for tasks like similarity search, clustering, and more.

In [ ]:
%pip install llama-index
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install llama-index-embeddings-openai

In [ ]:
!pip install openai


Traceback (most recent call last):
  File "/usr/local/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_cli.py", line 129, in main
    _main()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_cli.py", line 225, in _main
    parsed.func()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_api/models.py", line 43, in list
    models = get_client().models.list()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_utils.py", line 24, in get_client
    return _load_client()
  File "/usr/local/lib/python3.10/dist-packages/openai/__init__.py", line 327, in _load_client
    _client = _ModuleClient(
  File "/usr/local/lib/python3.10/dist-packages/openai/_client.py", line 105, in __init__
    raise OpenAIError(
openai.OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


Traceback (most recent call last):
  File "/usr/local/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_cli.py", line 129, in main
    _main()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_cli.py", line 225, in _main
    parsed.func()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_api/models.py", line 43, in list
    models = get_client().models.list()
  File "/usr/local/lib/python3.10/dist-packages/openai/cli/_utils.py", line 24, in get_client
    return _load_client()
  File "/usr/local/lib/python3.10/dist-packages/openai/__init__.py", line 327, in _load_client
    _client = _ModuleClient(
  File "/usr/local/lib/python3.10/dist-packages/openai/_client.py", line 105, in __init__
    raise OpenAIError(
openai.OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


## OpenAI API Key

Our frist step is to set our OpenAI API key.  If you are not sharing your notebook then you can simply set it here.  

    os.environ['OPENAI_API_KEY'] = 'OPENAI API KEY'

However, its advise to set it as an enviornment variable like so

    api_key = os.environ.get("OPENAI_API_KEY")

    [link text](https://)

Check out the "Set up a virtual environment" in the [OpenAI Quick start guide](https://platform.openai.com/docs/quickstart)  

In [ ]:
import google.colab
from google.colab import userdata
import os
import openai

## Get the API key from Co-lab Secrets
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key
print(f"API Key preview: {os.environ.get('OPENAI_API_KEY')[:5]}...")


API Key preview: sk-8k...


In [ ]:
# List off all the OpenAI Models
!openai api models.list

{
  "id": "gpt-4-turbo",
  "created": 1712361441,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "gpt-4o-mini-2024-07-18",
  "created": 1721172717,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "gpt-4-turbo-2024-04-09",
  "created": 1712601677,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "tts-1",
  "created": 1681940951,
  "object": "model",
  "owned_by": "openai-internal"
}
{
  "id": "tts-1-1106",
  "created": 1699053241,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "chatgpt-4o-latest",
  "created": 1723515131,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "dall-e-2",
  "created": 1698798177,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "gpt-4-turbo-preview",
  "created": 1706037777,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "gpt-3.5-turbo-instruct",
  "created": 1692901427,
  "object": "model",
  "owned_by": "system"
}
{
  "id": "gpt-4-0125-preview",
  "created": 1706037612,
  "object": "model",
  "owned_by"

In [ ]:
import os

from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import BaseTool, FunctionTool


The **llama_index.core** import statement brings in specific classes and functions from the llama_index.core module:

- StorageContext: This class is likely used to manage the context for storing data, such as configurations or settings for how data should be stored and retrieved.
- load_index_from_storage: This function is used to load an index from a storage context. An index is typically a data structure that improves the speed of data retrieval operations.

From the llama_index.core.tools module, we importn

- **BaseTool**  serves as a base class for creating various tools that can be used within the Llama Index framework. The purpose of a base class like BaseTool is to provide a common structure and set of functionalities that other, more specific tool classes can inherit and build upon.
- **FunctionTool** is designed to create tools from functions, encapsulating the functions with additional features or configurations that make them easier to manage and integrate into larger systems.

## Functions Definition
Here we define the functions themselves that will be used as tools.
We first start by a simple multiply functions that multiplies two numbers together.

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

We then wrap this function inside a FunctionTool object so that it can be used by the agent.

In [ ]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)

The FunctionTool.from_defaults method initializes a FunctionTool instance with default settings based on the provided function, enabling the function to be used as a standardized tool within the Llama Index framework.


## Setup ReAct Agent

New we setup a ReAct Agent (ReActAgent) and pass the tool we created to it.  :

ReActAgent is designed to implement a particular type of AI agent that follows the ReAct (Reflect, Act) paradigm. This paradigm involves an agent that can reflect on its actions and the environment before deciding on the next action, making it suitable for complex decision-making tasks.

The **OpenAI** class we've imprtanted can interface with OpenAI's large language models (LLMs) for various natural language processing (NLP) tasks, such as text generation, summarization, translation, and more.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [ ]:
# Check OpenAI Key is valid and set

# Initialize the OpenAI LLM with your API key implicitly set.
llm = OpenAI(model="gpt-3.5-turbo")  # or any other model you have access to

# Try creating a simple agent and running a basic query.
try:
  agent = ReActAgent.from_tools([], llm=llm)
  response = agent.chat("Hello!")
  print("API key is valid and working!")
except openai.error.AuthenticationError:
  print("API key is invalid or not set up correctly.")
except Exception as e:
  print(f"An error occurred: {e}")

API key is valid and working!


In [ ]:
# create a collectin of tools
tools = [multiply_tool]

In [ ]:
# [Optional] Add Context
context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """
llm = OpenAI(model="gpt-4o")

agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    context=context
)

This commented-out section defines an optional context that can be provided to the agent. The context sets a specific persona and expertise for the agent. The context is optional and can be included to influence the behavior and responses of the agent.

Lets create an instance of the OpenAI class

      llm = OpenAI(model="gpt-4o")

The model parameter specifies which version of OpenAI's language model to use. In this case, it's "gpt-4o".
   
Then we created an instance of the ReActAgent class using the from_tools method. This method configures the agent with the provided tools and language model.

- tools: A list of tools (in this case, it contains the multiply_tool) that the agent can use to perform specific tasks.
- llm=llm: The language model instance created earlier is passed to the agent, allowing it to use the capabilities of the gpt-4o model for natural language processing tasks.
- verbose=True: enable detailed logging
- context=context: The optional context is commented out. If uncommented, this context would be provided to the agent to influence its responses, making it act as a "stock market sorcerer" with expertise in Lyft and Uber.



In [ ]:
# interact with the AI agent by sending a query and receiving a response.
response = agent.chat("What is 2 times 8?")
print(str(response))

> Running step 1c362323-4e78-4fde-9c2d-58de37323a10. Step input: What is 2 times 8?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 8}
Observation: 16
> Running step a5ab7625-23b7-487c-af76-b5f5ed10a076. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2 times 8 is 16.
2 times 8 is 16.


### Output explainer:

Language Detection and Reasoning:

- Thought: The agent recognizes that the query is in English. This is a part of the agent's reasoning process, where it assesses the context and content of the query.
- Decision to Not Use Tools: The agent determines that the query is simple enough to be answered directly without invoking any additional tools. It concludes that the multiplication of two numbers (2 and 8) can be handled by the language model itself.

## More complex functions
As previously mentioned, we can build arbitrary functions and pass them to the agent as tools, which would enable the agent to basically perform any task we need as long as it can defined inside a function.

By default, the agent and the LLM it's using does not have access to the internet. But we can overcome that by defining the right functions.


This Python function uses the requests library to retrieve the content of a webpage given its URL. Here’s a detailed breakdown:

      import request
      ....
      response = requests.get(url)

The requests library is a powerful and user-friendly HTTP library in Python. It is used to send all kinds of HTTP requests, including GET requests to retrieve data from a web server.


In [ ]:
# prompt: build a python function that uses the requests library to retrieve a webpage and returns it

import requests

def fetch_webpage(url):
  """
  This function takes a URL as input and returns the webpage content as a string.
  """
  response = requests.get(url)
  if response.status_code == 200:
    return response.text
  else:
    raise Exception("Failed to fetch webpage.")


Lets get some movie reviews from Rotten Tomatoes

In [ ]:
def fetch_critics_reviews(movie_name):
  """
  This function takes a movie name as input and returns a list of reviews for that movie.
  """
  movie_name = movie_name.lower().replace(" ", "_")
  # url = f"https://www.imdb.com/title/{movie_name}/reviews?ref_=tt_urv"
  url = f"https://www.rottentomatoes.com/m/{movie_name}/reviews"
  reviews = fetch_webpage(url)
  return reviews


Our **fetch_critics_reviews** function retrieves reviews for a  movie from Rotten Tomatoes.  First we formats the movie name to be URL-friendly by converting it to lowercase and replacing spaces with underscores and then constructs a URL using movie name to call the reviews page on Rotten Tomatoes. We'll use the previously defined helper function, fetch_webpage, to send an HTTP GET request to the constructed URL and get the webpage content which is raw HTML btw.

Testing the function manually

In [ ]:
raw_html = fetch_critics_reviews('the dark knight')

In [ ]:
# If you want to see the actual formated output we can use BeautifulSoup...Uncomment this function however its quite verbose!!

'''
from bs4 import BeautifulSoup

def format_html(html_content):
    """
    This function takes raw HTML content as input and returns a nicely formatted string.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.prettify()

formatted_html = format_html(raw_html)
print(formatted_html)

'''

'\nfrom bs4 import BeautifulSoup\n\ndef format_html(html_content):\n    """\n    This function takes raw HTML content as input and returns a nicely formatted string.\n    """\n    soup = BeautifulSoup(html_content, \'html.parser\')\n    return soup.prettify()\n\nformatted_html = format_html(raw_html)\nprint(formatted_html)\n\n'

In [ ]:
# Wrap the function in a FunctionTool object
reviews_fetcher_tool = FunctionTool.from_defaults(fn=fetch_critics_reviews, description="This function takes a movie name as input and returns an HTML page containing the critics reviews for that movie.")


## Building a Movie Sentiment Analysis Agent

Lets demonstrates the effectiveness of AI agents in combining web scraping, natural language processing & understand, and contextual understanding to provide insightful and valuable responses.

Lets build an AI agent that can determine the general sentiment of a movie based on reviews from Rotten Tomatoes (movie review site). We'll integrate a custom tool that fetches reviews and calling OpenAI's natural language understandng (NLU) model, our agent can accurately analyze and summarize sentiment from text.

Now we define the agent and pass to it the tool we created.
We also give some context to the agent in the form of a prompt that explains its purpose.

In [ ]:
context = """\
You are an assistant that helps determine the general sentiment about a movie, given its critics reviews as HTML.
You have access to a tool that retrieves the critics reviews for a movie, given the movie name.
"""

llm = OpenAI(model="gpt-4o")

tools = [reviews_fetcher_tool]

agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    context=context
)

In [ ]:
# Use the tool in the agent's chat
response = agent.chat("What is the sentiment about the movie The Dark Knight?")
print(response)

> Running step 87165917-ad14-4ec3-96a0-462b986f257c. Step input: What is the sentiment about the movie The Dark Knight?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: fetch_critics_reviews
Action Input: {'movie_name': 'The Dark Knight'}
Observation: <!DOCTYPE html>
<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
        
        
            <script
                charset="UTF-8"
                crossorigin="anonymous"
                data-domain-script="7e979733-6841-4fce-9182-515fac69187f"
                integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN"
                src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js"
                type="te

This basically allows use to give our agent access to the internet and retrieve up to date information.
We test this by asking about a newer movie, that the LLM on its own would not know about.

In [ ]:
response = agent.chat("What is the sentiment about the movie Inside Out 2?")
print(response)

> Running step bc4daf9c-2b71-46f2-8f7d-2cc15e0ccdf8. Step input: What is the sentiment about the movie Inside Out 2?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: fetch_critics_reviews
Action Input: {'movie_name': 'Inside Out 2'}
Observation: <!DOCTYPE html>
<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
        
        
            <script
                charset="UTF-8"
                crossorigin="anonymous"
                data-domain-script="7e979733-6841-4fce-9182-515fac69187f"
                integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN"
                src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js"
                type="text/jav